In [32]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

IMG_SIZE = 160
BATCH_SIZE = 16

train_dir = "/kaggle/input/teeth-ds/teeth_data_set/Training"
val_dir = "/kaggle/input/teeth-ds/teeth_data_set/Validation"

# =========================
# Data Augmentation (صح)
# =========================
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,  # بدل rescale

    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.15,
    brightness_range=[0.8,1.2],
    fill_mode="nearest"
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input  # بدل rescale
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

NUM_CLASSES = train_gen.num_classes

# =========================
# Model
# =========================
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.45)(x)  # مهم للأسنان
output = Dense(NUM_CLASSES, activation="softmax")(x)

model = Model(base_model.input, output)

# =========================
# Compile
# =========================
model.compile(
    optimizer=Adam(learning_rate=3e-5),  # أقل شوية
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

Found 3702 images belonging to 7 classes.
Found 1058 images belonging to 7 classes.


In [33]:
dummy = tf.random.normal([1,160,160,3])
model(dummy)

<tf.Tensor: shape=(1, 7), dtype=float32, numpy=
array([[0.11605404, 0.11926621, 0.31626377, 0.08307325, 0.08005494,
        0.1469504 , 0.13833748]], dtype=float32)>

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Callbacks 
callbacks = [

    EarlyStopping(
        monitor="val_accuracy",
        patience=4,
        restore_best_weights=True,
        verbose=1
    ),

    ReduceLROnPlateau(
        monitor="val_loss",
        factor=0.3,
        patience=2,
        min_lr=1e-6,
        verbose=1
    ),

    ModelCheckpoint(
        "best_teeth_model.keras",
        monitor="val_accuracy",
        save_best_only=True,
        verbose=1
    )
]

# Phase 1 (Feature Extraction)
base_model.trainable = False

model.compile(
    optimizer=Adam(learning_rate=3e-5), 
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,      
    callbacks=callbacks,
    verbose=1
)

# Phase 2 (Fine-Tuning)
base_model.trainable = True

for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.1862 - loss: 2.0906
Epoch 1: val_accuracy improved from -inf to 0.28922, saving model to best_teeth_model.keras
232/232 ━━━━━━━━━━━━━━━━━━━━ 65s 199ms/step - accuracy: 0.1863 - loss: 2.0904 - val_accuracy: 0.2892 - val_loss: 1.8258 - learning_rate: 3.0000e-05
Epoch 2/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - accuracy: 0.2478 - loss: 1.8991
Epoch 2: val_accuracy improved from 0.28922 to 0.35822, saving model to best_teeth_model.keras
232/232 ━━━━━━━━━━━━━━━━━━━━ 27s 117ms/step - accuracy: 0.2477 - loss: 1.8990 - val_accuracy: 0.3582 - val_loss: 1.6902 - learning_rate: 3.0000e-05
Epoch 3/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step - accuracy: 0.3037 - loss: 1.8168
Epoch 3: val_accuracy improved from 0.35822 to 0.40454, saving model to best_teeth_model.keras
232/232 ━━━━━━━━━━━━━━━━━━━━ 28s 119ms/step - accuracy: 0.3037 - loss: 1.8167 - val_accuracy: 0.4045 - val_loss: 1.5874 - learning_rate: 3.0000e-05
Epoch 4/15

In [ ]:
history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30, 
    callbacks=callbacks
)

Epoch 1/30
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.8061 - loss: 0.6447
Epoch 1: val_accuracy improved from 0.84783 to 0.84972, saving model to best_teeth_model.keras
232/232 ━━━━━━━━━━━━━━━━━━━━ 28s 121ms/step - accuracy: 0.8061 - loss: 0.6447 - val_accuracy: 0.8497 - val_loss: 0.4767 - learning_rate: 1.0000e-05
Epoch 2/30
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.8105 - loss: 0.6202
Epoch 2: val_accuracy improved from 0.84972 to 0.85917, saving model to best_teeth_model.keras
232/232 ━━━━━━━━━━━━━━━━━━━━ 28s 122ms/step - accuracy: 0.8105 - loss: 0.6202 - val_accuracy: 0.8592 - val_loss: 0.4546 - learning_rate: 1.0000e-05
Epoch 3/30
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.8140 - loss: 0.6138
Epoch 3: val_accuracy improved from 0.85917 to 0.86578, saving model to best_teeth_model.keras
232/232 ━━━━━━━━━━━━━━━━━━━━ 28s 120ms/step - accuracy: 0.8141 - loss: 0.6137 - val_accuracy: 0.8658 - val_loss: 0.4298 - learning_rate: 1.0000e-05
Epoch 4

In [ ]:
history_fine = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=15, 
    callbacks=callbacks
)

Epoch 1/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.9500 - loss: 0.2273
Epoch 1: val_accuracy improved from 0.97732 to 0.97921, saving model to best_teeth_model.keras
232/232 ━━━━━━━━━━━━━━━━━━━━ 30s 128ms/step - accuracy: 0.9500 - loss: 0.2273 - val_accuracy: 0.9792 - val_loss: 0.1084 - learning_rate: 1.0000e-05
Epoch 2/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.9460 - loss: 0.2173
Epoch 2: val_accuracy improved from 0.97921 to 0.98110, saving model to best_teeth_model.keras
232/232 ━━━━━━━━━━━━━━━━━━━━ 30s 127ms/step - accuracy: 0.9460 - loss: 0.2173 - val_accuracy: 0.9811 - val_loss: 0.1014 - learning_rate: 1.0000e-05
Epoch 3/15
232/232 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.9439 - loss: 0.2198
Epoch 3: val_accuracy improved from 0.98110 to 0.98204, saving model to best_teeth_model.keras
232/232 ━━━━━━━━━━━━━━━━━━━━ 29s 127ms/step - accuracy: 0.9439 - loss: 0.2198 - val_accuracy: 0.9820 - val_loss: 0.0966 - learning_rate: 1.0000e-05
Epoch 4

KeyboardInterrupt: 

In [42]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

preds = best_model.predict(test_gen)
y_pred = np.argmax(preds, axis=1)

print(classification_report(test_gen.classes, y_pred))

34/34 ━━━━━━━━━━━━━━━━━━━━ 13s 201ms/step
              precision    recall  f1-score   support

           0       0.99      0.96      0.97        80
           1       0.99      1.00      0.99        76
           2       1.00      1.00      1.00        71
           3       0.97      0.96      0.97        80
           4       0.93      0.97      0.95        70
           5       0.95      0.96      0.96        80
           6       1.00      0.97      0.99        76

    accuracy                           0.98       533
   macro avg       0.98      0.98      0.98       533
weighted avg       0.98      0.98      0.98       533

